Source
- https://github.com/easonnie/combine-FEVER-NSMN
- https://github.com/likejazz/Siamese-LSTM
- https://blog.mlreview.com/implementing-malstm-on-kaggles-quora-question-pairs-competition-8b31b0b16a07
- Tahapan NSMN: https://docs.google.com/document/d/1Mu5p8rJMO-3kTs3Be4tHmJIoP7Yqylsim1Wx7IuW1zU/edit

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd '/content/drive/MyDrive/NSSN'

/content/drive/MyDrive/NSSN


In [3]:
!sudo apt-get update
!sudo apt-get install make
!sudo apt-get install gcc
!sudo apt-get install g++
!sudo apt-get install libffi-dev
!sudo apt-get install python-dev

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [2,858 kB]
Get:14 https://r2u.stat.illinois

In [4]:
# !pip install pyqt5==5.12
# !pip install pyqtwebengine==5.12
!pip install wrapt==1.12.1
# !pip install cffi==1.11.2
!pip install jsonnet
!pip install sqlitedict
!pip install wget
!pip install flashtext
!pip install pexpect
!pip install fire
!pip install inflection
!pip install folium==0.2.1
!pip install allennlp

  Preparing metadata (setup.py) ... done
  Created wheel for wrapt: filename=wrapt-1.12.1-cp310-cp310-linux_x86_64.whl size=71448 sha256=92dfd23b7b3cd26cd6693a4cdbcfdb0e9a4cb2f58c6e864ee7c35e1dd97cae06
  Stored in directory: /root/.cache/pip/wheels/8e/61/d3/d9e7053100177668fa43216a8082868c55015f8706abd974f2
Successfully built wrapt
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.14.1
    Uninstalling wrapt-1.14.1:
      Successfully uninstalled wrapt-1.14.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.2/594.2 kB 13.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for jsonnet: filename=jsonnet-0.20.0-cp310-cp310-linux_x86_64.whl size=6406863 sha256=95d843212fb2f41a2a696fbb7de1f69a14126aa310e4b9af5b9e7c47d582497b
  Stored in directory: /root/.cache/pip/wheels/63/0d/6b/5467dd1db9332ba4bd5cf4153e2870c5f89bb4db473d989cc2
Successfully built jsonnet
  Preparing metadata (setup.py) ... done
  Created wheel for sqlitedict: filename=sqlitedi

In [5]:
!source setup.sh
!bash ./scripts/prepare.sh

/bin/bash: line 1: setup.sh: No such file or directory
bash: ./scripts/prepare.sh: No such file or directory


In [6]:
from time import time
import pandas as pd

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Embedding, LSTM, GRU, Conv1D, Conv2D, GlobalMaxPool1D, Dense, Dropout

# Util
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Layer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from gensim.models import KeyedVectors
import gensim
import numpy as np
import itertools
import regex as re
from PIL import Image

In [7]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
def text_to_word_list(text):
    # Pre process and convert texts to a list of words
    text = str(text)
    text = text.lower()

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)

    text = text.split()

    return text


def make_w2v_embeddings(df, embedding_dim=300, empty_w2v=False):
    vocabs = {}
    vocabs_cnt = 0

    vocabs_not_w2v = {}
    vocabs_not_w2v_cnt = 0

    # Stopwords
    stops = set(stopwords.words('english'))

    # Load word2vec
    print("Loading word2vec model")

    if empty_w2v:
        word2vec = EmptyWord2Vec
    else:
        word2vec = KeyedVectors.load_word2vec_format("./data/GoogleNews-vectors-negative300.bin.gz", binary=True)
        # word2vec = gensim.models.word2vec.Word2Vec.load("./data/Quora-Question-Pairs.w2v").wv

    for index, row in df.iterrows():
        # Print the number of embedded sentences.
        if index != 0 and index % 1000 == 0:
            print("{:,} sentences embedded.".format(index), flush=True)

        # Iterate through the text of both questions of the row
        for question in ['claim', 'evidence']:

            q2n = []  # q2n -> question numbers representation
            for word in text_to_word_list(row[question]):
                # Check for unwanted words
                if word in stops:
                    continue

                # If a word is missing from word2vec model.
                if word not in word2vec.vocab:
                    if word not in vocabs_not_w2v:
                        vocabs_not_w2v_cnt += 1
                        vocabs_not_w2v[word] = 1

                # If you have never seen a word, append it to vocab dictionary.
                if word not in vocabs:
                    vocabs_cnt += 1
                    vocabs[word] = vocabs_cnt
                    q2n.append(vocabs_cnt)
                else:
                    q2n.append(vocabs[word])

            # Append question as number representation
            df.at[index, question + '_n'] = q2n

    embeddings = 1 * np.random.randn(len(vocabs) + 1, embedding_dim)  # This will be the embedding matrix
    embeddings[0] = 0  # So that the padding will be ignored

    # Build the embedding matrix
    for word, index in vocabs.items():
        if word in word2vec.vocab:
            embeddings[index] = word2vec.word_vec(word)
    del word2vec

    return df, embeddings


def split_and_zero_padding(df, max_seq_length):
    # Split to dicts
    X = {'left': df['claim_n'], 'right': df['evidence_n']}

    # Zero padding
    for dataset, side in itertools.product([X], ['left', 'right']):
        dataset[side] = pad_sequences(dataset[side], padding='pre', truncating='post', maxlen=max_seq_length)

    return dataset


#  --

class ManDist(Layer):
    """
    Keras Custom Layer that calculates Manhattan Distance.
    """

    # initialize the layer, No need to include inputs parameter!
    def __init__(self, **kwargs):
        self.result = None
        super(ManDist, self).__init__(**kwargs)

    # input_shape will automatic collect input shapes to build layer
    def build(self, input_shape):
        super(ManDist, self).build(input_shape)

    # This is where the layer's logic lives.
    def call(self, x, **kwargs):
        self.result = K.exp(-K.sum(K.abs(x[0] - x[1]), axis=1, keepdims=True))
        return self.result

    # return output shape
    def compute_output_shape(self, input_shape):
        return K.int_shape(self.result)


class EmptyWord2Vec:
    """
    Just for test use.
    """
    vocab = {}
    word_vec = {}

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [9]:
# Load training set
# train_df = pd.read_csv('data/train.csv')
train_df = pd.read_csv('data/fever.10000.train.csv')
for q in ['claim', 'evidence']:
    train_df[q + '_n'] = train_df[q]

In [22]:
train_df

,id,claim,evidence,label,topic,claim_n,evidence_n
0,1,Colin Kaepernick became a starting quarterback...,Colin Kaepernick became a starting quarterback...,1,sports,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]"
1,2,Tilda Swinton is a vegan.,Tilda Swinton is a vegan.,1,gossip,"[12, 13, 14]","[12, 13, 14]"
2,3,Anne Rice was born in New Jersey.,Anne Rice was born in New Jersey.,1,gossip,"[15, 16, 17, 18, 19]","[15, 16, 17, 18, 19]"
3,4,Happiness in Slavery is a gospel song by Nine ...,Happiness in Slavery is a gospel song by Nine ...,1,creation,"[20, 21, 22, 23, 24, 25, 26]","[20, 21, 22, 23, 24, 25, 26]"
4,5,Shooter is about an expert marksman who tries ...,Shooter is about an expert marksman who tries ...,1,creation,"[27, 28, 29, 30, 31, 32, 33]","[27, 28, 29, 30, 31, 32, 33]"
...,...,...,...,...,...,...,...
6995,6996,There are zero Polynesian languages.,Youtube is not a website.,0,sports,"[1905, 277, 278]","[2201, 3006]"
6996,6997,Wildfang was founded in orange.,Youtube is ranked as the fourth most popular s...,0,politics,"[247, 274, 5246]","[2201, 1531, 1532, 621, 3642, 115]"
6997,6998,The Good Wife is not a drama.,Youtube is unranked.,0,gossip,"[2910, 740, 682]","[2201, 5350]"
6998,6999,Stripes was directed by John Larroquette.,Youtube is unranked.,0,sports,"[625, 56, 135, 3516]","[2201, 5350]"


In [10]:
# Make word2vec embeddings
embedding_dim = 300
max_seq_length = 20
use_w2v = False

train_df, embeddings = make_w2v_embeddings(train_df, embedding_dim=embedding_dim, empty_w2v=not use_w2v)

Loading word2vec model
1,000 sentences embedded.
2,000 sentences embedded.
3,000 sentences embedded.
4,000 sentences embedded.
5,000 sentences embedded.
6,000 sentences embedded.


In [11]:
# Split to train validation
validation_size = int(len(train_df) * 0.1)
training_size = len(train_df) - validation_size

X = train_df[['claim_n', 'evidence_n']]
Y = train_df['label']

X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=validation_size)

X_train = split_and_zero_padding(X_train, max_seq_length)
X_validation = split_and_zero_padding(X_validation, max_seq_length)

# Convert labels to their numpy representations
Y_train = Y_train.values
Y_validation = Y_validation.values

# Make sure everything is ok
assert X_train['left'].shape == X_train['right'].shape
assert len(X_train['left']) == len(Y_train)

# --

In [12]:
# Model variables
gpus = 2
batch_size = 16 * gpus
n_epoch = 50
n_hidden = 10
# Define the shared model
x = Sequential()
x.add(Embedding(len(embeddings), embedding_dim, weights=[embeddings], input_shape=(max_seq_length,), trainable=False))
# LSTM
x.add(LSTM(n_hidden))
# DNN
x.add(Dense(5, activation='relu'))
x.add(Dropout(0.2))

shared_model = x

# The visible layer
left_input = Input(shape=(max_seq_length,), dtype='int32')
right_input = Input(shape=(max_seq_length,), dtype='int32')

# Pack it all up into a Manhattan Distance model
malstm_distance = ManDist()([shared_model(left_input), shared_model(right_input)])
model = Model(inputs=[left_input, right_input], outputs=[malstm_distance])

# model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])
model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(), metrics=['acc', f1_m, precision_m, recall_m])
model.summary()
shared_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 20)]                 0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 20)]                 0         []                            
                                                                                                  
 sequential (Sequential)     (None, 5)                    1618095   ['input_1[0][0]',             
                                                                     'input_2[0][0]']             
                                                                                                  
 man_dist (ManDist)          (None, 1)                    0         ['sequential[0][0]',      

In [13]:
# Start trainings
training_start_time = time()
malstm_trained = model.fit([X_train['left'], X_train['right']], Y_train,
                           batch_size=batch_size, epochs=n_epoch,
                           validation_data=([X_validation['left'], X_validation['right']], Y_validation))
training_end_time = time()
print("Training time finished.\n%d epochs in %12.2f" % (n_epoch,
                                                        training_end_time - training_start_time))

model.save('results/siamese/SiameseLSTM.h5')

Epoch 1/50
197/197 [==============================] - 10s 12ms/step - loss: 0.1271 - acc: 0.8192 - f1_m: 0.8624 - precision_m: 0.8637 - recall_m: 0.8679 - val_loss: 0.0590 - val_acc: 0.9157 - val_f1_m: 0.9374 - val_precision_m: 0.8858 - val_recall_m: 1.0000
Epoch 2/50
197/197 [==============================] - 1s 7ms/step - loss: 0.1000 - acc: 0.8744 - f1_m: 0.9030 - precision_m: 0.9090 - recall_m: 0.9011 - val_loss: 0.0576 - val_acc: 0.9243 - val_f1_m: 0.9431 - val_precision_m: 0.8955 - val_recall_m: 1.0000
Epoch 3/50
197/197 [==============================] - 2s 10ms/step - loss: 0.0860 - acc: 0.8951 - f1_m: 0.9176 - precision_m: 0.9316 - recall_m: 0.9075 - val_loss: 0.0618 - val_acc: 0.9100 - val_f1_m: 0.9338 - val_precision_m: 0.8794 - val_recall_m: 1.0000
Epoch 4/50
197/197 [==============================] - 2s 10ms/step - loss: 0.0800 - acc: 0.9079 - f1_m: 0.9287 - precision_m: 0.9344 - recall_m: 0.9263 - val_loss: 0.0580 - val_acc: 0.9171 - val_f1_m: 0.9384 - val_precision_m: 0.

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [14]:
# Load test set
test_df = pd.read_csv('data/fever.10000.test.csv')
for q in ['claim', 'evidence']:
    test_df[q + '_n'] = test_df[q]

# Make word2vec embeddings
embedding_dim = 300
max_seq_length = 20
test_df, embeddings = make_w2v_embeddings(test_df, embedding_dim=embedding_dim, empty_w2v=True)

# Split to dicts and append zero padding.
X_test = split_and_zero_padding(test_df, max_seq_length)

# Make sure everything is ok
assert X_test['left'].shape == X_test['right'].shape

prediction = model.predict([X_test['left'], X_test['right']])
print("\nCLAIM VERIFICATION RESULTS")


for pred in prediction:
  if(pred >= 0.56 and pred <= 1.0): print(str(pred) + " :Supports")
  elif(pred >= 0.0 and pred <= 0.44): print(str(pred) + " :Refutes")
  else:  print(str(pred) + " :Not Enough Information")

Loading word2vec model
1,000 sentences embedded.
2,000 sentences embedded.
94/94 [==============================] - 1s 3ms/step

CLAIM VERIFICATION RESULTS
[1.] :Supports
[1.] :Supports
[1.] :Supports
[1.] :Supports
[1.] :Supports
[1.] :Supports
[1.] :Supports
[1.] :Supports
[1.] :Supports
[1.] :Supports
[1.] :Supports
[1.] :Supports
[1.] :Supports
[1.] :Supports
[1.] :Supports
[1.] :Supports
[1.] :Supports
[1.] :Supports
[1.] :Supports
[1.] :Supports
[1.] :Supports
[1.] :Supports
[1.] :Supports
[1.] :Supports
[1.] :Supports
[1.] :Supports
[1.] :Supports
[1.] :Supports
[1.] :Supports
[1.] :Supports
[1.] :Supports
[1.] :Supports
[1.] :Supports
[1.] :Supports
[1.] :Supports
[1.] :Supports
[1.] :Supports
[1.] :Supports
[1.] :Supports
[1.] :Supports
[1.] :Supports
[1.] :Supports
[1.] :Supports
[1.] :Supports
[1.] :Supports
[1.] :Supports
[1.] :Supports
[1.] :Supports
[1.] :Supports
[1.] :Supports
[1.] :Supports
[1.] :Supports
[1.] :Supports
[1.] :Supports
[1.] :Supports
[1.] :Supports
[1.]

In [15]:
classified_predictions = []
for pred in prediction.flatten():  # Asumsikan 'prediction' adalah array numpy
    if 0.56 <= pred <= 1.0:
        classified_predictions.append(1)  # Supports
    elif 0.0 <= pred <= 0.44:
        classified_predictions.append(0)  # Refutes
    else:
        classified_predictions.append(2)  # Not Enough Information


In [16]:
# Mengubah classified_predictions menjadi one-hot encoding
one_hot_predictions = np.zeros((len(classified_predictions), 3))  # 3 kelas
for i, pred_class in enumerate(classified_predictions):
    one_hot_predictions[i, pred_class] = 1

In [17]:
Y_test = test_df['label'].values

In [18]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Asumsikan Y_test sudah dalam format one-hot encoding
predicted_labels = np.argmax(one_hot_predictions, axis=1)  # Mengonversi prediksi dari one-hot ke label kelas

conf_matrix = confusion_matrix(predicted_labels, Y_test)

# Menampilkan confusion matrix
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='g', cmap='Blues', xticklabels=['Refutes', 'Supports', 'NEI'], yticklabels=['Refutes', 'Supports', 'NEI'])
plt.ylabel('True Labels')
plt.xlabel('Predicted Labels')
plt.title('Confusion Matrix')
plt.show()
conf_matrix

array([[ 365,    0,    0],
       [ 560, 2000,    0],
       [  75,    0,    0]])

In [23]:
claim = "Alexandra Daddario is a ballerina"
evidence_sentences = train_df['evidence']

# Preprocess the claim
claim_n = text_to_word_list(claim)

# Create vocabulary from all sentences
all_sentences = [claim] + evidence_sentences
vocabs = {}
word_index = 1
for sentence in all_sentences:
    for word in text_to_word_list(sentence):
        if word not in vocabs:
            vocabs[word] = word_index
            word_index += 1

# Convert the claim to numerical representation
claim_n = [vocabs[word] if word in vocabs else 0 for word in claim_n]
claim_n = pad_sequences([claim_n], padding='pre', truncating='post', maxlen=max_seq_length)

predictions_for_evidence = []
for evidence in evidence_sentences:
    # Preprocess the evidence
    evidence_n = text_to_word_list(evidence)

    # Convert the evidence to numerical representation
    evidence_n = [vocabs[word] if word in vocabs else 0 for word in evidence_n]
    evidence_n = pad_sequences([evidence_n], padding='pre', truncating='post', maxlen=max_seq_length)

    # Predict the relationship between the claim and evidence
    prediction = model.predict([claim_n, evidence_n])
    predictions_for_evidence.append((prediction[0][0], evidence))

# Sort the predictions by confidence
predictions_for_evidence.sort(reverse=True)

# Print the top prediction and its corresponding evidence
top_prediction, top_evidence = predictions_for_evidence[0]

print("\nCLAIM VERIFICATION RESULT")
print("Claim:", claim)
print("Evidence:", top_evidence)
if 0.56 <= top_prediction <= 1.0:
    print(str(top_prediction) + " :Supports")
elif 0.0 <= top_prediction <= 0.44:
    print(str(top_prediction) + " :Refutes")
else:
    print(str(top_prediction) + " :Not Enough Information")

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 28ms/step

CLAIM VERIFICATION RESULT
Claim: Alexandra Daddario is a ballerina
Evidence: Youtube is ranked as the second most popular site in the world.
1.0 :Supports
